# **Distribución de créditos por grupos de edad**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "vallarta" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_vallarta',
 'df_financiamientos_2020_vallarta',
 'df_financiamientos_2021_vallarta',
 'df_financiamientos_2022_vallarta',
 'df_financiamientos_2023_vallarta',
 'df_financiamientos_2024_vallarta']

In [2]:
df_financiamientos_2024_vallarta.columns

Index(['año', 'mes', 'cve_ent', 'entidad', 'cve_mun', 'municipio', 'organismo',
       'modalidad', 'destino', 'tipo', 'sexo', 'edad_rango', 'ingresos_rango',
       'vivienda_valor', 'acciones', 'monto', 'id'],
      dtype='object')

In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id', 'edad_rango']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [4]:
dataframes_procesados = []

for df in dataframes_list:
    print(df['edad_rango'].unique())
    df = df[~(df['edad_rango']==0)]
    df.loc[:,'edad_rango'] = df['edad_rango'].replace({
        1:'<29',
        2:'30-59',
        3:'60<'
    })
    print(df['edad_rango'].unique())
    dataframes_procesados.append(df)

# Crea el DataFrame final concatenando todos los DataFrames procesados
df_final = pd.concat(dataframes_procesados, ignore_index=True)
df_final 

[0 1 2 3]
['<29' '30-59' '60<']
[1 2 3 0]
['<29' '30-59' '60<']
[1 2 3 0]
['<29' '30-59' '60<']
[0 1 2 3]
['<29' '30-59' '60<']
[1 2 3 0]
['<29' '30-59' '60<']
[2 0 1 3]
['30-59' '<29' '60<']


C:\Users\yoe11\AppData\Local\Temp\ipykernel_19160\1523741278.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['<29' '<29' '30-59' ... '30-59' '60<' '60<']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:,'edad_rango'] = df['edad_rango'].replace({
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19160\1523741278.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['<29' '<29' '30-59' ... '60<' '60<' '60<']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:,'edad_rango'] = df['edad_rango'].replace({
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19160\1523741278.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['<29' '30-59' '30-59' ... '<29' '30-59' '30-59']' has dtype incompatib

id edad_rango
0        2        <29
1        3        <29
2        4      30-59
3        5      30-59
4        6      30-59
...    ...        ...
8875  1176        60<
8876  1177      30-59
8877  1178      30-59
8878  1179      30-59
8879  1180        <29

[8880 rows x 2 columns]

In [7]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

def graficar_tabla(data, age_column, title):
    # Contar la cantidad de registros por cada rango de edad
    age_counts = data[age_column].value_counts()
    
    fig = go.Figure(data=[go.Pie(
        labels=age_counts.index,
        values=age_counts,
        marker=dict(colors=colores),
        textinfo='percent',
        insidetextorientation='radial',
        showlegend=False  # Ocultar la leyenda
    )])
    
    fig.update_layout(    
        yaxis=dict(
            # title='Rango de Edad', 
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        ),
        legend=dict(
            font_size=10,
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
            ),
        
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=1, r=1, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
    )    
    
    fig.show()
    return fig

# Suponiendo que ya tienes una lista de DataFrames llamada `dataframes_list`
fig1 = graficar_tabla(df_final, 'edad_rango', '')

In [8]:
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Save the graph as HTML and PNG
guardar_grafico_como_html(fig1, 'g_pie_creditos_edad', carpeta='assets/graficas')